# Airbnb analysis for European capital cities
## 1. Introduction
Airbnb has posted the listing data for various cities across the world on http://insideairbnb.com/get-the-data.html. In this project I will be analyzing the data to find insights to housing prices across major cities in Europe.

In [19]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [86]:
import warnings
warnings.filterwarnings('ignore')

In [188]:
import re

## 2. Data Gathering
Airbnb has posted the listing data for various cities across the world on http://insideairbnb.com/get-the-data.html. I have downloaded the data to my local hard drive. The data is in the form of a compressed csv file.

In [41]:
# Listing all the folders with different city information stored in the local hard drive

path = r'C:\Users\srini\Projects\Airbnb'
folders= os.listdir(path)
folders

['Amsterdam',
 'Berlin',
 'Brussels',
 'Copenhagen',
 'Lisbon',
 'London',
 'Madrid',
 'Oslo',
 'Paris',
 'Rome',
 'Stockholm']

In [31]:
# Importing the listing information for Amsterdam
df= pd.read_csv(os.path.join(path,folders[0], 'listings.csv.gz'))
df.head(2)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2818,https://www.airbnb.com/rooms/2818,20200508171622,2020-05-09,Quiet Garden View Room & Super Fast WiFi,Quiet Garden View Room & Super Fast WiFi,I'm renting a bedroom (room overlooking the ga...,Quiet Garden View Room & Super Fast WiFi I'm r...,none,"Indische Buurt (""Indies Neighborhood"") is a ne...",...,t,f,strict_14_with_grace_period,f,f,1,0,1,0,2.05
1,9693,https://www.airbnb.com/rooms/9693,20200508171622,2020-05-09,Top Location on Canal (Center Flat),You will love your stay here. It is a beautifu...,This beautiful apartment in the heart of 17th ...,You will love your stay here. It is a beautifu...,none,You will be on a beautiful quite canal and wil...,...,t,f,moderate,f,f,1,1,0,0,0.45


In [37]:
df.shape

(19278, 106)

Since the data for each city has the same file name 'listings' it will be accessed from across all the folders and merged to form a master dataframe.

In [56]:
# Merging the listing across all cities to form a master dataframe
for i,city in enumerate(folders):
    try:
        if i==0:
            df= pd.read_csv(os.path.join(path,city, 'listings.csv.gz'), low_memory=False)
                                                # reading the csv file for Amsterdam
        else:
            df= pd.concat([df,pd.read_csv(os.path.join(path,city, 'listings.csv.gz'),low_memory=False)] ) 
                                                # joining the remaining files
            
    except:
        print(city)


In [55]:
# Sending the compiled file to a local hard drive
df.to_csv(os.path.join(path,'listing_master.csv'), index= False)
df.shape

(320401, 106)

The master dataframe consists of more than 300,000 rows and 106 columns.

## 3. Data Wrangling

In [51]:
# Listing all the values for the first row
for i,col_name in enumerate(df.columns):
    print(col_name, df.iloc[0,i])

id 2818
listing_url https://www.airbnb.com/rooms/2818
scrape_id 20200508171622
last_scraped 2020-05-09
name Quiet Garden View Room & Super Fast WiFi
summary Quiet Garden View Room & Super Fast WiFi
space I'm renting a bedroom (room overlooking the garden) in my apartment in Amsterdam,  The room is located to the east of the city centre in a quiet, typical Amsterdam neighbourhood the "Indische Buurt". Amsterdam’s historic centre is less than 15 minutes away by bike or tram. The features of the room are: - Twin beds (80 x 200 cm, down quilts and pillows)  - 2 pure cotton towels for each guest  - reading lamps - bedside table - wardrobe - table with chairs - tea and coffee making facilities - mini bar - alarm clock - Hi-Fi system with cd player, connection for mp3 player / phone - map of Amsterdam and public transport - Wi-Fi Internet connection  Extra services: - Bike rental
description Quiet Garden View Room & Super Fast WiFi I'm renting a bedroom (room overlooking the garden) in my apa

### 3.1 Essential feature selection
Only some columns are essential for our current analysis. We will make a subset of our dataframe with the essential columns. For example columns with the house discription or picture of the house or the host are not essential for our analysis here and will be ignored.

In [59]:
# Making a list for essential columns
essential_columns= ['id', 'name', 'host_id','host_since', 'host_response_time','host_response_rate','host_acceptance_rate',\
'host_is_superhost','host_total_listings_count','host_verifications','street','city','country',\
 'property_type','room_type','accommodates','bathrooms','bedrooms','beds','bed_type','amenities','square_feet',\
'price','weekly_price','monthly_price','security_deposit','cleaning_fee','guests_included','extra_people','minimum_nights',\
'maximum_nights','availability_30','availability_60','availability_90','availability_365', 'number_of_reviews', \
'number_of_reviews_ltm','first_review','last_review','review_scores_rating','review_scores_accuracy',\
'review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location',\
'review_scores_value','is_business_travel_ready','cancellation_policy','require_guest_profile_picture','require_guest_phone_verification',\
'calculated_host_listings_count','calculated_host_listings_count_entire_homes',\
'calculated_host_listings_count_private_rooms','calculated_host_listings_count_shared_rooms','reviews_per_month']


I manually saw that the above columns would be essential for our Airbnb analysis and kept them.

In [61]:
# Making a subset dataframe with essential columns
df = df[essential_columns]

# Creating a backup for the cleaned dataframe
df.to_csv(os.path.join(path,'listing_master_clean.csv'), index= False)

df.shape

(320401, 55)

In [123]:
# Reading the csv 
df= pd.read_csv(os.path.join(path,'listing_master_clean.csv'))

### 3.2 Duplicate Values

In [124]:
# Finding duplicate values in the dataset
df.duplicated().sum()

0

In [125]:
# Finding if there are any duplicate IDs in the dataset
df.duplicated(subset= 'id', ).sum()

0

### 3.3 Removing the default index
Since we merged several dataframes together there are several rows with the same index number. These have to be removed.

In [126]:
# removing the default index 
df.reset_index(inplace= True)

In [127]:
# dropping the index column
df.drop(columns='index', inplace= True)

### 3.3 Data Types

In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320401 entries, 0 to 320400
Data columns (total 55 columns):
id                                              320401 non-null int64
name                                            320087 non-null object
host_id                                         320401 non-null int64
host_since                                      320344 non-null object
host_response_time                              181771 non-null object
host_response_rate                              181771 non-null object
host_acceptance_rate                            235387 non-null object
host_is_superhost                               320344 non-null object
host_total_listings_count                       320344 non-null float64
host_verifications                              320401 non-null object
street                                          320401 non-null object
city                                            319552 non-null object
country                               

**3.3.1 Date format for Host_since**

In [129]:
# changing the data type for host_since column
df.host_since= pd.to_datetime(df.host_since, )
type(df.host_since[0])

pandas._libs.tslibs.timestamps.Timestamp

**3.3.2 Float for response_rate column**

In [130]:
# exploring the response rate column
df.host_response_rate.unique()

array(['100%', nan, '50%', '90%', '0%', '89%', '70%', '83%', '80%', '22%',
       '20%', '75%', '67%', '13%', '33%', '40%', '86%', '94%', '30%',
       '78%', '98%', '29%', '71%', '25%', '60%', '57%', '88%', '10%',
       '63%', '97%', '95%', '44%', '54%', '43%', '91%', '92%', '93%',
       '56%', '17%', '79%', '38%', '41%', '85%', '62%', '73%', '87%',
       '64%', '96%', '99%', '82%', '81%', '76%', '14%', '69%', '31%',
       '11%', '47%', '27%', '55%', '74%', '72%', '42%', '58%', '84%',
       '77%', '52%', '46%', '66%', '68%', '53%', '37%', '36%', '51%',
       '15%', '61%', '6%', '9%', '8%', '5%', '65%'], dtype=object)

In [131]:
# Filling missing values with an arbitary number for further processessing
df.host_response_rate.fillna('0.1%', inplace= True)
df.host_response_rate.unique()

array(['100%', '0.1%', '50%', '90%', '0%', '89%', '70%', '83%', '80%',
       '22%', '20%', '75%', '67%', '13%', '33%', '40%', '86%', '94%',
       '30%', '78%', '98%', '29%', '71%', '25%', '60%', '57%', '88%',
       '10%', '63%', '97%', '95%', '44%', '54%', '43%', '91%', '92%',
       '93%', '56%', '17%', '79%', '38%', '41%', '85%', '62%', '73%',
       '87%', '64%', '96%', '99%', '82%', '81%', '76%', '14%', '69%',
       '31%', '11%', '47%', '27%', '55%', '74%', '72%', '42%', '58%',
       '84%', '77%', '52%', '46%', '66%', '68%', '53%', '37%', '36%',
       '51%', '15%', '61%', '6%', '9%', '8%', '5%', '65%'], dtype=object)

In [132]:
# changing the response_rate column to a decimal number
df.host_response_rate= df.host_response_rate.apply(lambda x: float(x[:-1])/100)

In [133]:
# Filling back the missing values
df.host_response_rate.replace({0.001:np.nan}, inplace= True)
df.host_response_rate.unique()

array([1.  ,  nan, 0.5 , 0.9 , 0.  , 0.89, 0.7 , 0.83, 0.8 , 0.22, 0.2 ,
       0.75, 0.67, 0.13, 0.33, 0.4 , 0.86, 0.94, 0.3 , 0.78, 0.98, 0.29,
       0.71, 0.25, 0.6 , 0.57, 0.88, 0.1 , 0.63, 0.97, 0.95, 0.44, 0.54,
       0.43, 0.91, 0.92, 0.93, 0.56, 0.17, 0.79, 0.38, 0.41, 0.85, 0.62,
       0.73, 0.87, 0.64, 0.96, 0.99, 0.82, 0.81, 0.76, 0.14, 0.69, 0.31,
       0.11, 0.47, 0.27, 0.55, 0.74, 0.72, 0.42, 0.58, 0.84, 0.77, 0.52,
       0.46, 0.66, 0.68, 0.53, 0.37, 0.36, 0.51, 0.15, 0.61, 0.06, 0.09,
       0.08, 0.05, 0.65])

**3.3.3 Float for acceptance_rate column**

In [141]:
# Taking similar steps for host_acceptance_rate column
df.host_acceptance_rate.fillna('0.1%', inplace= True)
df.host_acceptance_rate= df.host_acceptance_rate.apply(lambda x: float(x[:-1])/100)
df.host_acceptance_rate.replace({0.001:np.nan}, inplace= True)
df.host_acceptance_rate.unique()

array([1.  , 0.97, 0.53, 0.99,  nan, 0.73, 0.35, 0.96, 0.95, 0.2 , 0.55,
       0.98, 0.78, 0.89, 0.67, 0.4 , 0.64, 0.57, 0.  , 0.59, 0.93, 0.47,
       0.88, 0.94, 0.42, 0.75, 0.5 , 0.62, 0.83, 0.9 , 0.76, 0.38, 0.82,
       0.71, 0.56, 0.32, 0.33, 0.7 , 0.14, 0.92, 0.72, 0.81, 0.86, 0.23,
       0.84, 0.3 , 0.8 , 0.44, 0.58, 0.25, 0.17, 0.36, 0.39, 0.63, 0.09,
       0.69, 0.77, 0.46, 0.27, 0.6 , 0.21, 0.79, 0.68, 0.26, 0.91, 0.19,
       0.85, 0.65, 0.07, 0.29, 0.13, 0.01, 0.45, 0.43, 0.52, 0.41, 0.22,
       0.87, 0.48, 0.24, 0.15, 0.31, 0.37, 0.28, 0.61, 0.18, 0.08, 0.66,
       0.12, 0.54, 0.02, 0.49, 0.16, 0.11, 0.74, 0.06, 0.51, 0.1 , 0.05,
       0.34, 0.03, 0.04])

**3.3.4 Boolean values for is_superhost column**

In [145]:
# Replacing t and f values as 1 and 0 in the host_is_superhost column
df.host_is_superhost.replace({'t':1, 'f':0}, inplace= True)
df.host_is_superhost.value_counts()

0.0    266905
1.0     53439
Name: host_is_superhost, dtype: int64

**3.3.5 Float values for price, weekly_price, monthly_price, cleaning_cost, secuirty deposit columns**

In [160]:
# Changing price to float value
def price_to_float(x):
    """Removes the ',' and '$' from the price value and returns a float"""
    try:
        x= x.replace(',','')
        x_transformed = float(x[1:])
        
    except:
        try:
            x_transformed = float(x[1:])
        except:
            x_transformed= None
        
    return x_transformed

In [158]:
# Changing price to float value
df.price= df.price.apply(lambda x: price_to_float(x))

type(df.price[0])

numpy.float64

In [162]:
# Changing weekly price to float value
df.weekly_price= df.weekly_price.apply(lambda x: price_to_float(x))
type(df.weekly_price[0])

numpy.float64

In [163]:
# Changing monthly price to float value
df.monthly_price= df.monthly_price.apply(lambda x: price_to_float(x))
type(df.monthly_price[0])

numpy.float64

In [165]:
# Changing security_fee to float value
df.security_deposit= df.security_deposit.apply(lambda x: price_to_float(x))
type(df.security_deposit[0])

numpy.float64

In [166]:
# Changing cleaning fee to float value
df.cleaning_fee= df.cleaning_fee.apply(lambda x: price_to_float(x))
type(df.cleaning_fee[0])

numpy.float64

In [169]:
# Changing extra people cost to float value
df.extra_people= df.extra_people.apply(lambda x: price_to_float(x))
type(df.extra_people[0])

numpy.float64

**3.3.6 Date format for first and last review columns**

In [171]:
# Changing to date type for first review and last review
df.first_review= pd.to_datetime(df.first_review)
df.last_review = pd.to_datetime(df.last_review)

**3.3.7 Boolean format for guest requirement columns**

In [177]:
# Changing require_guest_profile_picture to boolean values
df.require_guest_profile_picture.replace({'t':1, 'f':0}, inplace= True)
df.require_guest_profile_picture.value_counts()

0    317794
1      2607
Name: require_guest_profile_picture, dtype: int64

In [178]:
# Changing require_guest_phone_verification to boolean values
df.require_guest_phone_verification.replace({'t':1, 'f':0}, inplace= True)
df.require_guest_phone_verification.value_counts()

0    314904
1      5497
Name: require_guest_phone_verification, dtype: int64

In [179]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320401 entries, 0 to 320400
Data columns (total 54 columns):
id                                              320401 non-null int64
name                                            320087 non-null object
host_id                                         320401 non-null int64
host_since                                      320344 non-null datetime64[ns]
host_response_time                              181771 non-null object
host_response_rate                              181771 non-null float64
host_acceptance_rate                            235387 non-null float64
host_is_superhost                               320344 non-null float64
host_total_listings_count                       320344 non-null float64
host_verifications                              320401 non-null object
street                                          320401 non-null object
city                                            319552 non-null object
country                    

### 3.3 Missing Values

In [180]:
df.isna().sum()

id                                                   0
name                                               314
host_id                                              0
host_since                                          57
host_response_time                              138630
host_response_rate                              138630
host_acceptance_rate                             85014
host_is_superhost                                   57
host_total_listings_count                           57
host_verifications                                   0
street                                               0
city                                               849
country                                              0
property_type                                        0
room_type                                            0
accommodates                                         0
bathrooms                                          290
bedrooms                                           475
beds      

In [135]:
# Proportion of missing values in Square feet column
df.square_feet.isna().sum()/ df.shape[0]

0.9872721995249703

In [136]:
# Proportion of missing values in the weekly price column
1- df.weekly_price.isna().sum()/ df.shape[0]

0.10609205339558869

In [137]:
# Proportion of missing values in the monthly price column
1- df.monthly_price.isna().sum()/ df.shape[0]

0.07262773836536096

Since the square feet column has only 1.3% of values we can drop this column as most of the Airbnb hosts do not seem to have filled this column.

In [138]:
# Dropping the square_feet column
df.drop(columns='square_feet', inplace= True)
df.shape

(320401, 54)

The weekly price column and the monthly price column have only 7-10% filled. This means that the host did not have a special price for week or month and the regular price was only used for calculating them. 
Hence, we will fill pricex7 and pricex30 for weekly and monthly missing values.

In [183]:
# Filling weekly and monthly price missing values using the daily price
df.weekly_price.fillna(df.price*7, inplace= True)

df.monthly_price.fillna(df.price*30, inplace= True)

Where there is a missing value for the security fee and the cleaning fee we can assume that to be zero as the host has not mentioned it.

In [185]:
# Filling zero value for missing security deposit fee and the cleaning fee
df.security_deposit.fillna(0, inplace= True)
df.cleaning_fee.fillna(0, inplace= True)

### 3.5 Modifying Amenities column

In [202]:
# Findina all possible amenties across 300,000 listings
amenities_list=[]

def amenities_list_func(x):
    """Function to list all the amenities"""
    
    split_list = x[1:-1].split(',') #splitting list seperated by commas
    split_list_modified=[]
    
    for item in split_list:
        
        try:
            item= item.strip('\"')  # Removing the "" char from some of the amenities
            split_list_modified.append(item)
        except:
            split_list_modified.append(item)
            
            
        if item not in amenities_list:
            
            amenities_list.append(item) # appending the item list
    
    return split_list_modified


In [209]:
# Adding a new column with cleaned up values of amenties
df.amenities_modified= df.amenities.apply(lambda x: amenities_list_func(x))

In [206]:
len(amenities_list)

296

In [210]:
df.amenities_modified[0]

['Internet',
 'Wifi',
 'Paid parking off premises',
 'Buzzer/wireless intercom',
 'Heating',
 'Washer',
 'Smoke detector',
 'Carbon monoxide detector',
 'First aid kit',
 'Safety card',
 'Fire extinguisher',
 'Essentials',
 'Shampoo',
 'Lock on bedroom door',
 '24-hour check-in',
 'Hangers',
 'Hair dryer',
 'Iron',
 'Laptop friendly workspace',
 'translation missing: en.hosting_amenity_49',
 'translation missing: en.hosting_amenity_50',
 'Private entrance',
 'Hot water',
 'Bed linens',
 'Extra pillows and blankets',
 'Ethernet connection',
 'Coffee maker',
 'Refrigerator',
 'Single level home',
 'Garden or backyard',
 'Host greets you',
 'Paid parking on premises',
 'Trash can']

There are a total of 296 seperate amenities listed in all the listings. A seperate dataframe needs to be made that has this information for all the listings.

In [237]:
# Making a dictionary that has 296 keys for each item in the amenities list. It appends its values based on the fact 
#  if the item is in that df row or not
amenities_dic= {}
row=0
def amenities_dic_func(x):
    """ Makes a dictionary that has 296 keys for each item in the amenities list. Its values are appened to boolean
    if that df row contains that amenity item in it or not"""
    global row
    for am_item in amenities_list:

        if row==0:   # for the first element of the dictonary
            if am_item in x:
                amenities_dic[am_item]=[1]
            else:
                amenities_dic[am_item]=[0]
        else:      # for the subsequent elements of the dictonary
            if am_item in x:
                amenities_dic[am_item].append(1)
            else:
                amenities_dic[am_item].append(0)
    row +=1

In [238]:
# Making a dictionary for each row of the amenities column
df.amenities_modified.apply(lambda x: amenities_dic_func(x))

0         None
1         None
2         None
3         None
4         None
          ... 
320396    None
320397    None
320398    None
320399    None
320400    None
Name: amenities, Length: 320401, dtype: object

In [239]:
amenities_df=  pd.DataFrame(amenities_dic)
amenities_df.shape

(320401, 296)

In [240]:
# Creating a backup for the amenities df
amenities_df.to_csv(os.path.join(path,'amenities_df.csv'), index= False)

## 4. Exploratory Data Analysis

In [241]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320401 entries, 0 to 320400
Data columns (total 54 columns):
id                                              320401 non-null int64
name                                            320087 non-null object
host_id                                         320401 non-null int64
host_since                                      320344 non-null datetime64[ns]
host_response_time                              181771 non-null object
host_response_rate                              181771 non-null float64
host_acceptance_rate                            235387 non-null float64
host_is_superhost                               320344 non-null float64
host_total_listings_count                       320344 non-null float64
host_verifications                              320401 non-null object
street                                          320401 non-null object
city                                            319552 non-null object
country                    